In [1]:
# Setup and install necessary libraries
!pip install langchain_community sentence-transformers pymupdf chromadb transformers faiss-cpu PyPDF2 pdfplumber

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.0/42.0 kB 3.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 7.0 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.6/50.6 kB 5.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 48.5/48.5 kB 4.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 82.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 255.8/255.8 kB 22.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 19.6/19.6 MB 97.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 607.0/607.0 kB 45.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 90.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 27.5/27.5 MB 76.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 232.6/232.6 kB 20.2 MB/s et

In [2]:
# Mount Google Drive (if using in Colab)
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
# Import necessary libraries
import fitz
import pdfplumber
from langchain.text_splitter import RecursiveCharacterTextSplitter
from sentence_transformers import SentenceTransformer

/usr/local/lib/python3.10/dist-packages/sentence_transformers/cross_encoder/CrossEncoder.py:13: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm, trange


In [4]:
# Function to load PDF text using pymupdf
def load_pdf_with_pymupdf(file_path):
    try:
        text = ""
        with fitz.open(file_path) as pdf:
            for page in pdf:
                text += page.get_text()
        text = ' '.join(text.split())  # Clean up whitespace
        return text
    except Exception as e:
        print(f"Failed to load PDF using pymupdf: {e}")
        return ""

# Function to load PDF text using pdfplumber
def load_pdf_with_pdfplumber(file_path):
    try:
        text = ""
        with pdfplumber.open(file_path) as pdf:
            for page in pdf.pages:
                text += page.extract_text() if page.extract_text() else ""
        text = ' '.join(text.split())  # Clean up whitespace
        return text
    except Exception as e:
        print(f"Failed to load PDF using pdfplumber: {e}")
        return ""

In [5]:
# Choose one of the functions to load PDF text
file_path = "/content/drive/MyDrive/Colab Notebooks/IETM_sLM_ARG/Artificial Intelligence Industry Trend Brief.pdf"  # Example file path
pdf_text_pymupdf = load_pdf_with_pymupdf(file_path)
pdf_text_pdfplumber = load_pdf_with_pdfplumber(file_path)

In [6]:
# 두 방법의 출력을 비교
if pdf_text_pymupdf:
    print("Loaded text using pymupdf:", pdf_text_pymupdf[:500])
else:
    print("Loaded text using pdfplumber:", pdf_text_pdfplumber[:500])


Loaded text using pymupdf: 2023년 12월호 2023년 12월호 Ⅰ. 인공지능 산업 동향 브리프 1. 정책/법제 ▹ 미국, 안전하고 신뢰할 수 있는 AI 개발과 사용에 관한 행정명령 발표 ························· 1 ▹ G7, 히로시마 AI 프로세스를 통해 AI 기업 대상 국제 행동강령에 합의··························· 2 ▹ 영국 AI 안전성 정상회의에 참가한 28개국, AI 위험에 공동 대응 선언··························· 3 ▹ 미국 법원, 예술가들이 생성 AI 기업에 제기한 저작권 소송 기각····································· 4 ▹ 미국 연방거래위원회, 저작권청에 소비자 보호와 경쟁 측면의 AI 의견서 제출················· 5 ▹ EU AI 법 3자 협상, 기반모델 규제 관련 견해차로 난항··················································· 6 2. 기


In [7]:
# 로드된 텍스트를 추가 처리를 위해 청크로 분할합니다.
def split_text(text, chunk_size=1000, chunk_overlap=100):
    splitter = RecursiveCharacterTextSplitter(
        chunk_size=chunk_size,
        chunk_overlap=chunk_overlap,
        separators=["\\n\\n", "\\n", " ", ""]
    )
    split_texts = splitter.split_text(text)
    return split_texts


In [8]:
# 추출된 텍스트 사용 (필요에 따라 조정)
text_chunks = split_text(pdf_text_pymupdf if pdf_text_pymupdf else pdf_text_pdfplumber)
print(f"Number of chunks: {len(text_chunks)}")
print(text_chunks[:3])

Number of chunks: 31
['2023년 12월호 2023년 12월호 Ⅰ. 인공지능 산업 동향 브리프 1. 정책/법제 ▹ 미국, 안전하고 신뢰할 수 있는 AI 개발과 사용에 관한 행정명령 발표 ························· 1 ▹ G7, 히로시마 AI 프로세스를 통해 AI 기업 대상 국제 행동강령에 합의··························· 2 ▹ 영국 AI 안전성 정상회의에 참가한 28개국, AI 위험에 공동 대응 선언··························· 3 ▹ 미국 법원, 예술가들이 생성 AI 기업에 제기한 저작권 소송 기각····································· 4 ▹ 미국 연방거래위원회, 저작권청에 소비자 보호와 경쟁 측면의 AI 의견서 제출················· 5 ▹ EU AI 법 3자 협상, 기반모델 규제 관련 견해차로 난항··················································· 6 2. 기업/산업 ▹ 미국 프런티어 모델 포럼, 1,000만 달러 규모의 AI 안전 기금 조성································ 7 ▹ 코히어, 데이터 투명성 확보를 위한 데이터 출처 탐색기 공개 ······································· 8 ▹ 알리바바 클라우드, 최신 LLM ‘통이치엔원 2.0’ 공개 ······················································ 9 ▹ 삼성전자, 자체 개발 생성 AI ‘삼성 가우스’ 공개 ··························································· 10 ▹ 구글, 앤스로픽에 20억 달러 투자로 생성 AI 협력 강화 ················································ 11 ▹ IDC, 2027년 AI 소프트웨어 매출 2,500억 달러 돌파', 'AI

In [9]:
# SentenceTransformer를 사용하여 임베딩 생성
embedding_model = SentenceTransformer('jhgan/ko-sroberta-multitask')
def embed_text_chunks(text_chunks):
    embeddings = embedding_model.encode(text_chunks, convert_to_tensor=True, show_progress_bar=True)
    return embeddings

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/229 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/123 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/4.86k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

vocab.txt:   0%|          | 0.00/248k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/495k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/156 [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [10]:
# 텍스트 청크에 대한 임베딩 생성
text_embeddings = embed_text_chunks(text_chunks)
print(f"Number of embeddings: {len(text_embeddings)}")
print(text_embeddings[0])  # Example output of the first embedding

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Number of embeddings: 31
tensor([ 1.1058e-01,  5.4661e-01, -5.6452e-02, -5.0433e-01, -1.4841e-02,
        -2.3026e-01, -4.6539e-01, -5.0768e-04,  1.1751e-01,  1.1742e-01,
        -4.4150e-01, -1.2791e-01, -8.1903e-02,  6.9033e-01, -1.0353e-01,
         5.4862e-01, -3.7257e-01, -1.2612e-01, -1.9711e-01,  5.1191e-02,
        -7.6774e-02,  5.5403e-01, -1.1485e-01,  7.1137e-02,  2.2718e-01,
         3.6755e-01,  3.8852e-01, -7.7544e-02,  1.9370e-01,  5.7660e-01,
         2.5787e-01, -6.5055e-02, -3.9957e-01, -1.7972e-03, -5.3046e-02,
         2.7924e-02, -3.1045e-01, -1.0634e-01,  2.1354e-01, -1.7261e-01,
        -5.3218e-01, -1.4438e-01, -4.9238e-03,  1.4768e-01,  6.1912e-01,
         4.5830e-02, -1.5036e-01,  2.1606e-02,  1.9207e-01, -1.0361e-01,
         4.2816e-01, -1.8861e-01, -9.9749e-02,  2.7707e-01, -1.8360e-01,
        -3.9992e-02, -2.7900e-01,  4.0439e-01, -1.7546e-01,  1.9504e-01,
         2.5682e-02, -7.3458e-01,  1.3020e-01,  4.2527e-01,  4.0710e-01,
         2.4295e-02,  1.38

In [11]:
import chromadb
from chromadb.config import Settings

# ChromaDB 설정 및 클라이언트 초기화 (새로운 방식)
chroma_client = chromadb.Client(Settings(
    persist_directory="/content/drive/MyDrive/Colab Notebooks/chroma_db" # 데이터 저장 경로 설정
))

# 컬렉션 생성
collection_name = "ai_industry_trends"
if collection_name in chroma_client.list_collections():
    collection = chroma_client.get_collection(collection_name)
else:
    collection = chroma_client.create_collection(name=collection_name)

# 텍스트 조각을 벡터 임베딩과 함께 컬렉션에 추가하는 함수
def store_embeddings_in_chroma(text_chunks, embeddings):
    # 문서 ID를 생성
    ids = [f"doc_{i}" for i in range(len(text_chunks))]

    # 컬렉션에 데이터 추가 (documents 필드 추가)
    collection.add(
        ids=ids,
        embeddings=[embedding.tolist() for embedding in embeddings],  # 벡터를 리스트로 변환
        metadatas=[{"text": chunk} for chunk in text_chunks],           # 메타데이터에 원본 텍스트 저장
        documents=text_chunks                                           # documents 필드에 원본 텍스트 추가
    )
    print(f"Added {len(text_chunks)} documents to the ChromaDB collection.")


# 임베딩을 ChromaDB에 저장
store_embeddings_in_chroma(text_chunks, text_embeddings)


Added 31 documents to the ChromaDB collection.


In [12]:
from sentence_transformers import SentenceTransformer
import chromadb
from chromadb.config import Settings

# 사전 학습된 임베딩 모델 로드
embedding_model = SentenceTransformer('jhgan/ko-sroberta-multitask')

# ChromaDB 클라이언트 초기화
chroma_client = chromadb.Client(Settings(
    persist_directory="/content/drive/MyDrive/Colab Notebooks/chroma_db" # 이전에 설정한 경로와 동일
))

# 기존 컬렉션 가져오기
collection_name = "ai_industry_trends"
collection = chroma_client.get_collection(collection_name)

# 다양한 쿼리 표현을 생성하는 함수
def generate_diverse_queries(query):
    # 쿼리 변형 예제 (간단한 형태)
    variations = [
        query,
        f"How does {query} impact the industry?",
        f"Explain the trends in {query}",
        f"What are the latest updates in {query}?",
        f"Recent advancements in {query}",
    ]
    return variations

# 다양한 쿼리 기반으로 검색을 수행하는 Retriever
class DiverseQueryRetriever:
    def __init__(self, embedding_model, collection):
        self.embedding_model = embedding_model
        self.collection = collection

    def search(self, query, top_k=5):
        # 다양한 쿼리 생성
        diverse_queries = generate_diverse_queries(query)

        # 각 변형된 쿼리에 대해 임베딩 생성
        query_embeddings = self.embedding_model.encode(diverse_queries, convert_to_tensor=True)

        # 검색 결과 통합
        results = []
        for query_embedding in query_embeddings:
            search_results = self.collection.query(
                query_embeddings=[query_embedding.tolist()],
                n_results=top_k
            )

            # 검색 결과 디버그 메시지 출력
#            print("\n[디버그] 검색 결과:", search_results)

            # 검색 결과가 유효한지 확인하고 추가
            if search_results and 'documents' in search_results:
                documents = search_results.get('documents', [[]])
                if documents:
                    for doc_list in documents:
                        if isinstance(doc_list, list):
                            results.extend(doc_list)
                        else:
                            results.append(doc_list)


        # 중복 제거: 문자열 리스트에 대해 set 사용
        unique_results = list(set(results))
        return unique_results


# Retriever 인스턴스 생성
retriever = DiverseQueryRetriever(embedding_model, collection)

# 테스트: 다양한 쿼리로 문서 검색
query = "AI"
results = retriever.search(query)
print(results)


['by Countries Attending the AI Safety Summit, 1-2 November 2023, 2023.11.01. Gov.uk, World leaders, top AI companies set out plan for safety testing of frontier as first global AI Safety Summit concludes, 2023.11.02. SPRi AI Brief | 2023-12월호 4 미국 법원, 예술가들이 생성 AI 기업에 제기한 저작권 소송 기각 n 미국 캘리포니아 북부지방법원은 미드저니, 스태빌리티AI, 디비언트아트를 대상으로 예술가 3인이 제기한 저작권 침해 소송을 기각 n 법원은 기각 이유로 고소장에 제시된 상당수 작품이 저작권청에 등록되지 않았으며, AI로 생성된 이미지와 특정 작품 간 유사성을 입증하기 어렵다는 점을 제시 KEY Contents £ 예술가들의 AI 저작권 침해 소송, 저작권 미등록과 증거불충분으로 기각 n 미국 캘리포니아 북부지방법원의 윌리엄 오릭(William Orrick) 판사는 2023년 10월 30일 미드저니 (Midjourney), 스태빌리티AI(Stability AI), 디비언트아트(DeviantArt)에 제기된 저작권 침해 소송을 기각 ∙2023년 1월 예술가 사라 앤더슨(Sarah Anderson), 캘리 맥커넌(Kelly McKernan), 칼라 오르티즈(Karla Ortiz)는 이미지 생성 AI 서비스를 개발한 3개 기업을 상대로 저작권 침해 소송을 제기 ∙예술가들은 3개 기업이 AI 모델을 학습시키기 위해 원작자 동의 없이 작품을 학습 데이터셋에 포함하여 저작권을 침해했다고 주장했으며, 법원은 지난 4월 피소 기업들이 제출한 기각 신청을 수용해 소송을 기각 n 오릭 판사는 판결문에서 소송을 기각한 핵심 이유로 예술가들의 저작권 미등록을 제시 ∙판결문은 소송을 제기한 캘리 맥커넌과 칼라 오르티즈가 미국 저작권청에 예술 작품에 대한 저작권을 제출하지 않았다는 점을'

In [15]:
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer, pipeline
from sentence_transformers import SentenceTransformer
import chromadb
from chromadb.config import Settings

# 허깅페이스 모델과 토크나이저 로드
model_name = "KISTI-KONI/KONI-Llama3-8B-Instruct-20240729"
tokenizer = AutoTokenizer.from_pretrained(model_name)

# GPU 메모리 최적화를 위한 설정 (float16 + device_map)
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    torch_dtype=torch.float16,
    device_map="auto"  # 자동 메모리 최적화
)

# 텍스트 생성 파이프라인 설정
gpt_pipeline = pipeline("text-generation", model=model, tokenizer=tokenizer)

# 사전 학습된 임베딩 모델 로드
embedding_model = SentenceTransformer('jhgan/ko-sroberta-multitask')

# ChromaDB 클라이언트 초기화
chroma_client = chromadb.Client(Settings(
    persist_directory="/content/drive/MyDrive/Colab Notebooks/chroma_db" # 이전에 설정한 경로와 동일
))

# 기존 컬렉션 가져오기
collection_name = "ai_industry_trends"
collection = chroma_client.get_collection(collection_name)

# 텍스트 조각을 벡터 임베딩과 함께 컬렉션에 추가하는 함수
def store_embeddings_in_chroma(text_chunks, embeddings):
    ids = [f"doc_{i}" for i in range(len(text_chunks))]
    collection.add(
        ids=ids,
        embeddings=[embedding.tolist() for embedding in embeddings],
        metadatas=[{"text": chunk} for chunk in text_chunks],
        documents=text_chunks
    )
    print(f"Added {len(text_chunks)} documents to the ChromaDB collection.")

# 유사도 기반 검색을 수행하는 Retriever
class SimilarityQueryRetriever:
    def __init__(self, embedding_model, collection):
        self.embedding_model = embedding_model
        self.collection = collection

    def search(self, query, top_k=5):
        query_embedding = self.embedding_model.encode(query, convert_to_tensor=True)
        search_results = self.collection.query(
            query_embeddings=[query_embedding.tolist()],
            n_results=top_k
        )

        results = []
        if search_results and 'documents' in search_results:
            documents = search_results.get('documents', [[]])
            metadatas = search_results.get('metadatas', [[]])
            if documents:
                for doc_list, metadata_list in zip(documents, metadatas):
                    for doc, metadata in zip(doc_list, metadata_list):
                        if doc:
                            results.append({"id": metadata.get("id", ""), "text": doc, "metadata": metadata})

        return results

# 검색된 결과를 기반으로 프롬프트 작성
def generate_prompt_from_results(query, search_results, max_context_length=500):
    if not search_results:
        return None

    context = []
    for result in search_results[:3]:  # 상위 3개의 검색 결과만 사용
        if isinstance(result, dict) and 'text' in result:
            context.append(result['text'])
        elif isinstance(result, str):
            context.append(result)

    context = "\n".join(context)
    if len(context) > max_context_length:
        context = context[:max_context_length] + "..."  # 문맥이 길면 자르기

    return context

# 검색된 결과를 기반으로 답변 생성
def generate_answer(query, search_results, max_new_tokens=50):
    context = generate_prompt_from_results(query, search_results)
    if not context:
        return "관련된 정보를 찾을 수 없습니다. 다른 질문을 시도해 보세요."

    # 실제 답변 생성
    prompt = f"{context}\n\n답변:"
    response = gpt_pipeline(prompt, max_new_tokens=max_new_tokens, num_return_sequences=1, do_sample=True, temperature=0.5)

    # 출력에서 질문과 문맥 부분을 제거하고 실제 답변만 반환
    answer = response[0]['generated_text'].strip().replace(prompt, "").strip()
    return answer

# 대화 상태 관리 (이전 대화를 포함하여 문맥 유지)
class ChatBot:
    def __init__(self):
        self.retriever = SimilarityQueryRetriever(embedding_model, collection)

    def get_response(self, user_input):
        # 검색 수행
        results = self.retriever.search(user_input)
        # 이전 대화 내용과 새로운 검색 결과를 기반으로 답변 생성
        return generate_answer(user_input, results)

    def run(self):
        print("챗봇과 대화를 시작하세요. 종료하려면 'exit'을 입력하세요.")
        while True:
            user_input = input("\n사용자: ")
            if user_input.lower() == "exit":
                print("대화를 종료합니다.")
                break

            response = self.get_response(user_input)
            print(f"\n챗봇:{response}")

# 챗봇 실행
chatbot = ChatBot()
chatbot.run()


Loading checkpoint shards:   0%|          | 0/7 [00:00<?, ?it/s]

챗봇과 대화를 시작하세요. 종료하려면 'exit'을 입력하세요.

사용자: AI 트랜드에 대해서 알려줘


KeyboardInterrupt: 